In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15:
        continue
    
#     preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

처음에 어떻게 해야 텐서가 15개를 넘어가는 것을 제외해야 될지 몰라서 고민하며 파이썬 문법을 보다 보니
아주 간단하게 split() 메서드를 통해 분해할 수 있어서 사용했다

근데 이게 맞는지는 모르겠다.

In [4]:
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000, filters=' ', oov_token="<unknown>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f762ff32850>


In [6]:
print(tensor[:3, :10])
print(type(tensor))

[[   2   50    4   95  303   62   53    9  946 6269]
 [   2   15 2971  872    5    8   11 5747    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]
<class 'numpy.ndarray'>


In [7]:
source_input = tensor[:, :-1]  
target_input = tensor[:, 1:]    

enc_train, enc_val, dec_train, dec_val = train_test_split(source_input, target_input, test_size=0.2, random_state=43)

print("source train :", enc_train.shape)
print("Target train : ", dec_train.shape)

source train : (124981, 14)
Target train :  (124981, 14)


In [8]:
# BUFFER_SIZE = len(source_input)
# BATCH_SIZE = 256
# steps_per_epoch = len(source_input) // BATCH_SIZE

# VOCAB_SIZE = tokenizer.num_words + 1

# dataset = tf.data.Dataset.from_tensor_slices((source_input, target_input))
# dataset = dataset.shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
# dataset

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1000
hidden_size = 3000
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)


In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
3906/3906 [==============================] - 1036s 253ms/step - loss: 2.9822 - val_loss: 2.6886
Epoch 2/10
3906/3906 [==============================] - 990s 254ms/step - loss: 2.3941 - val_loss: 2.4223
Epoch 3/10
3906/3906 [==============================] - 990s 254ms/step - loss: 1.9008 - val_loss: 2.2827
Epoch 4/10
3906/3906 [==============================] - 990s 253ms/step - loss: 1.5275 - val_loss: 2.2453
Epoch 5/10
3906/3906 [==============================] - 990s 253ms/step - loss: 1.2893 - val_loss: 2.2814
Epoch 6/10
3906/3906 [==============================] - 990s 253ms/step - loss: 1.1551 - val_loss: 2.3298
Epoch 7/10
3906/3906 [==============================] - 990s 253ms/step - loss: 1.0885 - val_loss: 2.3810
Epoch 8/10
3906/3906 [==============================] - 989s 253ms/step - loss: 1.0506 - val_loss: 2.4266
Epoch 9/10
3906/3906 [==============================] - 989s 253ms/step - loss: 1.0289 - val_loss: 2.4446
Epoch 10/10
3906/3906 [======================

In [11]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl , <end> '